In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def sim_factor_model(loadings, specific_variance, mu, nsim=1, verbose=True):
    """
    Parameters
    ---
        loadings:           (p, k) matrix
        specific_variance:  (p, p) diagonal matrix, with specific variances on diagonals
        mu:                 (p, 1) vector of means
        nsim:               How many observations should be simulated

    Returns
    ---
        (n, p) matrix of observations from the specified factor model

    """
    k = loadings.shape[1]
    p = specific_variance.shape[0]
    if verbose:
        print(f"{k=} {p=}")

    X = []
    for _ in range(nsim):
        factor_vector = np.random.multivariate_normal(np.zeros(k), np.eye(k))
        u = np.random.multivariate_normal(np.zeros(p), np.diag(specific_variance))

        X.append(loadings @ factor_vector + u + mu)

    return np.array(X)

Goal is to minimize:
$$
tr((\hat{\Lambda}\hat{\Lambda}' + \Psi)^{-1} S) - log(|(\hat{\Lambda}\hat{\Lambda}' + \Psi)^{-1} S|) \quad \text{w.r.t. $\Psi$}
$$


Step by step:
1. Calculate $S^* = \Psi^{-1/2} S \Psi^{-1/2}$, $S^* = \Gamma \Theta \Gamma'$
2. Get eigenvectors $\gamma_{(i)}$ and values, $\theta_i$
3. Now $\Lambda^*$ has columns $c_i \lambda_{(i)}, \quad c_i = \sqrt{\max(\theta_i - 1, 0)}$
4. $\hat\Lambda = \Psi^{1/2} \Lambda^*$
5. Calculate the object function $\text{tr}((\hat\Lambda \hat\Lambda' + \Psi)^{-1} S) - log(|(\hat\Lambda \hat\Lambda' + \Psi)^{-1}S|)$

In [3]:
# Generate synthetic data
loadings = np.array([[2, 2, 2, 2, 2]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**4)
X

k=1 p=5


array([[ 6.65089651, 17.78540527, 26.70362514, 36.94176474, 46.97685246],
       [11.31736753, 19.20930876, 31.98769527, 41.08432567, 52.21852275],
       [10.86820111, 18.38365978, 34.28626804, 39.18370095, 50.83430591],
       ...,
       [ 8.66754372, 23.22831836, 36.07534928, 42.17754191, 50.79561654],
       [ 9.55792508, 20.52399331, 29.04923932, 39.31125254, 50.7793516 ],
       [ 8.88154953, 18.58262242, 31.51107332, 41.06549962, 49.95514229]])

1. Calculate $S^*$

In [4]:
S = np.cov(X.T)
Psi = np.diag(specific_variance)
Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
S_star = Psi_sq_inv @ S @ Psi_sq_inv

2. Get eigenvectors and eigenvalues

In [5]:
eigval, eigvec = np.linalg.eig(S_star)

3. Construct $\Lambda^*$.

Here we must choose k. Lets choose k = 1, like the underlying model that generated the data. (the `loadings` used to simulate is (5, 1) = (p, k))

In [6]:
lambda_star = max(eigval[0] - 1, 0) ** 0.5 * eigvec[:,0]
lambda_star

array([1.40958185, 1.40795684, 0.62943818, 2.0160881 , 1.99247972])

4. Contstruct $\hat\Lambda = \Psi^{1/2} \Lambda^*$

In [7]:
lambda_hat = Psi ** 0.5 @ lambda_star
lambda_hat

array([1.99344976, 1.99115166, 1.99045829, 2.0160881 , 1.99247972])

5. Calculate $\text{tr}((\hat\Lambda \hat\Lambda' + \Psi)^{-1} S) - log(|(\hat\Lambda \hat\Lambda' + \Psi)^{-1}S|)$

In [8]:
internal = np.linalg.inv(lambda_hat @ lambda_hat + Psi) @ S
result = np.trace(internal) - np.log(np.linalg.det(internal))
result

5.760287625659386

Having done this, lets construct a function that takes an arbitrary (D, 1) array of specific variances and calculates the objective function

In [9]:
def calculate_objective(specific_variance, X_data):
    # Step 1
    S = np.cov(X_data.T)
    Psi = np.diag(specific_variance)
    Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
    S_star = Psi_sq_inv @ S @ Psi_sq_inv

    # Step 2
    eigval, eigvec = np.linalg.eig(S_star)

    # Step 3
    lambda_star = max(eigval[0] - 1, 0) ** 0.5 * eigvec[:,0]

    # Step 4
    lambda_hat = Psi ** 0.5 @ lambda_star

    # Step 5
    internal = np.linalg.inv(lambda_hat @ lambda_hat.T + Psi) @ S
    result = np.trace(internal) - np.log(np.linalg.det(internal))

    return result

In [10]:
specific_variance = np.array([2, 2, 10, 1, 1])
calculate_objective(specific_variance, X)

5.760287625659386

This matches our manual step by step!

Now lets minimize the objective function. Note that the minimization algorithm works best with variables in $\R$, however $\psi_{ii} \geq 0$. We circumvent this by optimizing with $\alpha_i \in \R \rightarrow \psi_{ii} = \exp(\alpha_i) \in [0, \infty [$

In [11]:
x_0_guess = np.array([2, 2, 10, 1, 1])
problem = minimize(fun=lambda x: calculate_objective(np.exp(x), X_data=X),
                   x0=x_0_guess)

In [12]:
psi_hat = np.exp(problem.x)
psi_hat

array([ 1.98257924,  2.00912895, 10.21223486,  1.01941838,  0.96365588])

A simple change in step 3 lets us add the option to specify k number of factors.

In [13]:
def calculate_objective(specific_variance, X_data, k):
    # Step 1
    S = np.cov(X_data.T)
    Psi = np.diag(specific_variance)
    Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
    S_star = Psi_sq_inv @ S @ Psi_sq_inv

    # Step 2
    eigval, eigvec = np.linalg.eig(S_star)

    # Step 3
    lambda_star = []
    for i in range(k):
        lambda_star.append(max(eigval[i] - 1, 0) ** 0.5 * eigvec[:,i])
    lambda_star = np.array(lambda_star).T

    # Step 4
    lambda_hat = Psi ** 0.5 @ lambda_star

    # Step 5
    internal = np.linalg.inv(lambda_hat @ lambda_hat.T + Psi) @ S
    result = np.trace(internal) - np.log(np.linalg.det(internal))

    return result

Let's generate some data that actually has k = 2 factors.

In [14]:
loadings = np.array([[2, 2, 2, 2, 2],
                     [1, 1, 0, -1, -1]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**5)

k=2 p=5


In [16]:
x_0_guess = np.array([0, 0, 0, 0, 0])
problem = minimize(fun=lambda x: calculate_objective(np.exp(x), X_data=X, k=2),
         x0=x_0_guess)
problem

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 5.4294456018892765
        x: [ 1.563e+00  1.564e+00  1.571e-01 -8.485e-03  1.254e-02]
      nit: 11
      jac: [ 2.154e-03  2.145e-03 -7.815e-03 -9.997e-03 -1.012e-02]
 hess_inv: [[ 1.108e+00  1.082e-01 ... -2.969e-02 -8.903e-03]
            [ 1.082e-01  1.108e+00 ... -2.855e-02 -8.086e-03]
            ...
            [-2.969e-02 -2.855e-02 ...  2.406e+00  1.069e+00]
            [-8.903e-03 -8.086e-03 ...  1.069e+00  1.803e+00]]
     nfev: 606
     njev: 100

Then estimates are:

In [138]:
psi_hat = np.diag(np.exp(problem.x))

k = 2
S = np.cov(X.T)
Psi_sq_inv = np.linalg.inv(psi_hat ** 0.5)
S_star = Psi_sq_inv @ S @ Psi_sq_inv
eigval, eigvec = np.linalg.eig(S_star)
lambda_star = []
for i in range(2):
    lambda_star.append(max(eigval[i] - 1, 0) ** 0.5 * eigvec[:,i])
lambda_star = np.array(lambda_star).T
lambda_hat = psi_hat ** 0.5 @ lambda_star

print("Estimated psi: ", psi_hat, 
      "",
      "Estimated lambda:", lambda_hat,
      sep="\n")

Estimated psi: 
[[2.01971318 0.         0.         0.         0.        ]
 [0.         1.95827486 0.         0.         0.        ]
 [0.         0.         9.97996892 0.         0.        ]
 [0.         0.         0.         0.99084424 0.        ]
 [0.         0.         0.         0.         1.01165497]]

Estimated lambda:
[[ 1.75415117  1.36202321]
 [ 1.76979477  1.40110577]
 [ 1.95270123  0.40536224]
 [ 2.15384765 -0.59007522]
 [ 2.15466319 -0.58493958]]
